In [1]:
from datetime import date, datetime, timedelta
import pandas as pd

import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn import metrics, preprocessing
from sklearn.model_selection import KFold


from scipy import stats
import seaborn as sns

import os
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def obtain_natural_chunks(features, labels, terminals):
    feature_list = []
    label_list = []
    for i in range(len(terminals) - 1):
        idx = np.logical_and(features[:, 0] >= terminals[i], features[:, 0] < terminals[i + 1])
        feature_list.append(features[idx][:, 1:])
        label_list.append(labels[idx])
    return feature_list, label_list


In [4]:
def obtain_intervals(dataset):
    '''
    Generate interval terminals, so that samples in each interval have:
        interval_i = (timestamp >= terminal_i) and (timestamp < terminal_{i+1})

    Args:
        dataset (chr): Assuming only Backblaze (b) and Google (g) datasets exists
    '''
    if dataset == 'g':
        # time unit in Google: millisecond, tracing time: 29 days
        start_time = 604046279
        unit_period = 24 * 60 * 60 * 1000 * 1000  # unit period: one day
        end_time = start_time + 28*unit_period
    elif dataset == 'b':
        # time unit in Backblaze: week, tracing time: one year (50 weeks)
        start_time = 1
        unit_period = 7  # unit period: one week (7 days)
        end_time = start_time + 50*unit_period
    # original 1 month
    '''
    elif dataset == 'b':
        # time unit in Backblaze: month, tracing time: one year (12 months)
        start_time = 1
        unit_period = 1  # unit period: one month
        end_time = start_time + 12*unit_period
    '''
    
    
    # add one unit for the open-end of range function
    terminals = [i for i in range(start_time, end_time+unit_period, unit_period)]

    return terminals

# Reading Data

In [5]:
DATASET_PATH = '../../Documents/phd_related/data_sets_concept_drift/AIOps_failure_prediction/disk_failure_2015.csv'
interval = 'm'

In [6]:
features_disk_failure = ['smart_1_raw', 'smart_4_raw', 'smart_5_raw', 'smart_7_raw', 'smart_9_raw', 'smart_12_raw', 'smart_187_raw', 'smart_193_raw', 'smart_194_raw', 'smart_197_raw', 'smart_199_raw', 
                         'smart_4_raw_diff', 'smart_5_raw_diff', 'smart_9_raw_diff', 'smart_12_raw_diff', 'smart_187_raw_diff', 'smart_193_raw_diff', 'smart_197_raw_diff', 'smart_199_raw_diff']
columns = ['serial_number', 'date'] + features_disk_failure + ['label']

In [7]:
len(features_disk_failure)

19

In [8]:
df = pd.read_csv(DATASET_PATH, header=None)
# put columns names
df.columns = columns
# ignore serial number
df = df[df.columns[1:]]

In [9]:
# transform date to date time
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [10]:
# divide on days of year

# original implementation
#df['date'] = pd.Series(pd.DatetimeIndex(df['date']).month)

# divide on weeks
df['date'] = pd.Series(pd.DatetimeIndex(df['date']).day_of_year)

In [11]:
features = df[df.columns[:-1]].to_numpy()
labels = df[df.columns[-1]].to_numpy()

# Data Preprocessing

In [12]:
# DIVIDE FEATURES INTO WEEKS 

feature_list, label_list = obtain_natural_chunks(features, labels, obtain_intervals('b'))

In [13]:
len(feature_list)

50

# Drift Detection Accuracy Important Features Only

In [14]:
data_path_drift_localization = '../../Documents/phd_related/AIOps_disk_failure_prediction/feature_importance/rf_concept_drift_localization_disk_week_2015_r_'
random_seeds = ['1234', '4887', '597', '1959', '413', '44', '2969', '4971', '4913', '9591']

In [15]:
no_overall_correct_all = []
no_drift_correct_all = []
no_non_drift_correct_all = []
random_seed_all = []

important_features_count = 0
count = 0

for i in tqdm(range(1, 11)):
    df_concept_drift = pd.read_csv(data_path_drift_localization + str(i) + '.csv')
    df_concept_drift = df_concept_drift.loc[:, ~df_concept_drift.columns.str.contains('^Unnamed')]
    #df_concept_drift = df_concept_drift.reset_index(drop = True)
    drifts_true_disk = []
    for k in range(0, len(df_concept_drift.Sig.values)):
        if(df_concept_drift.Sig.values[k]==True):
            drifts_true_disk.append(1)
        else:
            drifts_true_disk.append(0)

    scaler = StandardScaler()
    stat_test = stats.kstest

    ede_ks_results_important_features = []

    for period in tqdm(range(0, len(feature_list)-1)):
    #for period in tqdm(range(1,2)):


        #print('Before Scale Train', feature_list[period])
        #print('Before Scale Test', feature_list[period+1])

        # extract features train and test
        training_features = scaler.fit_transform(feature_list[period])
        testing_features = scaler.transform(feature_list[period+1])

        # convert numpy array to Pandas Dataframe
        df_train_features = pd.DataFrame(training_features, columns = features_disk_failure)
        df_test_features = pd.DataFrame(testing_features, columns = features_disk_failure)

        # Sort by Feature Importance

        # Extract Ranks
        feature_imp_array_string = np.array(((df_concept_drift.FI[period].replace('[','')).replace(']','')).replace('\n', '').split(' '))
        #print('feature_imp_array_string', feature_imp_array_string)

        # convert to float
        feature_imp_array = [float(i) for i in feature_imp_array_string if i != '' ]

        # consider only the most important features (importance > mean(feature_importances))
        # extract mean?? of feature importance
        mean_importance = np.mean(feature_imp_array)

        zipped_features = list(zip(feature_imp_array, features_disk_failure))


        sorted_features_zip = sorted(zipped_features, key = lambda x: x[0], reverse = True)

        sorted_important_features = [x for x in sorted_features_zip if x[0]>=mean_importance]

        sorted_features = [i[1] for i in sorted_important_features]
        print('LEN SORTED', len(sorted_features))
        important_features_count = important_features_count + len(sorted_features)
        count = count + 1

        # Sort columns in df according to their importance
        df_train_features_sorted = df_train_features[sorted_features]
        df_test_features_sorted = df_test_features[sorted_features]


        distribution_training = sns.distplot(np.array(df_train_features_sorted)).get_lines()[0].get_data()[1]
        plt.close()
        distribution_test = sns.distplot(np.array(df_test_features_sorted)).get_lines()[0].get_data()[1]
        plt.close()

        v, p = stat_test(distribution_training, distribution_test)
        if(p<0.05):
            ede_ks_results_important_features.append(1)
        else:
            ede_ks_results_important_features.append(0)
    
    
    no_overall_correct = 0
    no_drift_correct = 0
    no_non_drift_correct = 0

    for j in tqdm(range(0, len(ede_ks_results_important_features))):

        if(ede_ks_results_important_features[j]==drifts_true_disk[j]):
            no_overall_correct = no_overall_correct + 1

        if(drifts_true_disk[j]==1):
            if(ede_ks_results_important_features[j]==drifts_true_disk[j]):
                no_drift_correct = no_drift_correct + 1

        if(drifts_true_disk[j]==0):
            if(ede_ks_results_important_features[j]==drifts_true_disk[j]):
                no_non_drift_correct = no_non_drift_correct + 1

    # Compute Metrics for all Random Seeds         
    overall_correct_prediction_score = no_overall_correct/len(drifts_true_disk)
    no_overall_correct_all.append(overall_correct_prediction_score)
    
    drift_correct_prediction_score = no_drift_correct/len(np.nonzero(drifts_true_disk)[0])
    no_drift_correct_all.append(drift_correct_prediction_score)
    
    non_drift_correct_prediction_score = no_non_drift_correct/(len(drifts_true_disk) - len(np.nonzero(drifts_true_disk)[0]))
    no_non_drift_correct_all.append(non_drift_correct_prediction_score)
    
    random_seed_all.append([random_seeds[i-1]]*len(feature_list))

  0%|          | 0/49 [00:00<?, ?it/s]

LEN SORTED 6



  2%|▏         | 1/49 [00:02<01:40,  2.10s/it]

LEN SORTED 7



  4%|▍         | 2/49 [00:04<01:47,  2.28s/it]

...


 96%|█████████▌| 47/49 [02:25<00:08,  4.01s/it]

LEN SORTED 7



 98%|█████████▊| 48/49 [02:30<00:04,  4.19s/it]

LEN SORTED 5



100%|██████████| 49/49 [02:33<00:00,  3.14s/it]

100%|██████████| 10/10 [24:22<00:00, 146.23s/it]


In [19]:
important_features_count/count

6.426530612244898

Avg accuracy of overall correct detected batches

In [16]:
np.mean(no_overall_correct_all)

0.6387755102040816

Avg accuracy of correct detected drifts

In [17]:
np.mean(no_drift_correct_all)

0.68972963092026

Avg accuracy of corrected detected non-drifts

In [18]:
np.mean(no_non_drift_correct_all)

0.44369241869241866

# Drift Detection Accuracy All Features

In [20]:
no_overall_correct_all = []
no_drift_correct_all = []
no_non_drift_correct_all = []
random_seed_all = []

for i in tqdm(range(1, 11)):
    df_concept_drift = pd.read_csv(data_path_drift_localization + str(i) + '.csv')
    df_concept_drift = df_concept_drift.loc[:, ~df_concept_drift.columns.str.contains('^Unnamed')]
    drifts_true_disk = []
    for k in range(0, len(df_concept_drift.Sig.values)):
        if(df_concept_drift.Sig.values[k]==True):
            drifts_true_disk.append(1)
        else:
            drifts_true_disk.append(0)

    scaler = StandardScaler()
    stat_test = stats.kstest

    ede_ks_results_important_features = []

    for period in tqdm(range(0, len(feature_list)-1)):
    #for period in tqdm(range(1,2)):


        #print('Before Scale Train', feature_list[period])
        #print('Before Scale Test', feature_list[period+1])

        # extract features train and test
        training_features = scaler.fit_transform(feature_list[period])
        testing_features = scaler.transform(feature_list[period+1])

        # convert numpy array to Pandas Dataframe
        df_train_features = pd.DataFrame(training_features, columns = features_disk_failure)
        df_test_features = pd.DataFrame(testing_features, columns = features_disk_failure)

        # Sort by Feature Importance

        # Extract Ranks
        feature_imp_array_string = np.array(((df_concept_drift.FI[period].replace('[','')).replace(']','')).replace('\n', '').split(' '))
        #print('feature_imp_array_string', feature_imp_array_string)

        # convert to float
        feature_imp_array = [float(i) for i in feature_imp_array_string if i != '' ]

        # consider only the most important features (importance > mean(feature_importances))
        # extract mean of feature importance
        mean = np.mean(feature_imp_array)

        zipped_features = list(zip(feature_imp_array, features_disk_failure))


        sorted_features_zip = sorted(zipped_features, key = lambda x: x[0], reverse = True)
        
        
        sorted_features = [i[1] for i in sorted_features_zip]
        
        df_train_features_sorted = df_train_features[sorted_features]
        df_test_features_sorted = df_test_features[sorted_features]
                
        
        distribution_training = sns.distplot(np.array(df_train_features_sorted)).get_lines()[0].get_data()[1]
        plt.close()
        distribution_test = sns.distplot(np.array(df_test_features_sorted)).get_lines()[0].get_data()[1]
        plt.close()

        v, p = stat_test(distribution_training, distribution_test)
        if(p<0.05):
            ede_ks_results_important_features.append(1)
        else:
            ede_ks_results_important_features.append(0)
    
    
    no_overall_correct = 0
    no_drift_correct = 0
    no_non_drift_correct = 0

    for j in tqdm(range(0, len(ede_ks_results_important_features))):

        if(ede_ks_results_important_features[j]==drifts_true_disk[j]):
            no_overall_correct = no_overall_correct + 1

        if(drifts_true_disk[j]==1):
            if(ede_ks_results_important_features[j]==drifts_true_disk[j]):
                no_drift_correct = no_drift_correct + 1

        if(drifts_true_disk[j]==0):
            if(ede_ks_results_important_features[j]==drifts_true_disk[j]):
                no_non_drift_correct = no_non_drift_correct + 1

    # Compute Metrics for all Random Seeds         
    overall_correct_prediction_score = no_overall_correct/len(drifts_true_disk)
    no_overall_correct_all.append(overall_correct_prediction_score)
    
    drift_correct_prediction_score = no_drift_correct/len(np.nonzero(drifts_true_disk)[0])
    no_drift_correct_all.append(drift_correct_prediction_score)
    
    non_drift_correct_prediction_score = no_non_drift_correct/(len(drifts_true_disk) - len(np.nonzero(drifts_true_disk)[0]))
    no_non_drift_correct_all.append(non_drift_correct_prediction_score)
    
    random_seed_all.append([random_seeds[i-1]]*len(feature_list))
        

100%|██████████| 49/49 [07:27<00:00,  9.13s/it]

100%|██████████| 49/49 [07:06<00:00,  8.69s/it]

100%|██████████| 49/49 [06:44<00:00,  8.25s/it]

100%|██████████| 49/49 [06:45<00:00,  8.28s/it]

100%|██████████| 49/49 [07:00<00:00,  8.57s/it]

100%|██████████| 49/49 [07:02<00:00,  8.62s/it]

 18%|█▊        | 9/49 [00:55<04:14,  6.36s/it]


100%|██████████| 49/49 [06:55<00:00,  8.48s/it]

100%|██████████| 49/49 [07:13<00:00,  8.85s/it]

100%|██████████| 49/49 [07:12<00:00,  8.82s/it]

100%|██████████| 49/49 [06:37<00:00,  8.11s/it]

100%|██████████| 10/10 [1:10:07<00:00, 420.71s/it]


Avg accuracy of overall correct detected batches

In [21]:
np.mean(overall_correct_prediction_score)

0.5714285714285714

Avg accuracy of correct detected drifts

In [22]:
np.mean(no_drift_correct_all)

0.7150090245982414

Avg accuracy of corrected detected non-drifts

In [23]:
np.mean(no_non_drift_correct_all)

0.08892968142968144